In [1]:

%pprint
%matplotlib inline
import sys
import os.path as osp

executable_path = sys.executable
scripts_folder = osp.join(osp.dirname(executable_path), 'Scripts'); assert osp.exists(scripts_folder)
py_folder = osp.abspath(osp.join('..', 'py')); assert osp.exists(py_folder), "Create the py folder"
ffmpeg_folder = r'C:\ffmpeg\bin'; assert osp.exists(ffmpeg_folder)
shared_folder = osp.abspath(osp.join('..', 'share')); assert osp.exists(shared_folder)

if (scripts_folder not in sys.path): sys.path.insert(1, scripts_folder)
if (py_folder not in sys.path): sys.path.insert(1, py_folder)
if (ffmpeg_folder not in sys.path): sys.path.insert(1, ffmpeg_folder)
if shared_folder not in sys.path: sys.path.insert(1, shared_folder)
    
from notebook_utils import NotebookUtilities
nu = NotebookUtilities(
    data_folder_path=osp.abspath(osp.join('..', 'data')),
    saves_folder_path=osp.abspath(osp.join('..', 'saves'))
)

# Import needed libraries
import os, re
import pandas as pd
import pyperclip
import ipywidgets as widgets
from IPython.display import display

Pretty printing has been turned OFF


In [4]:

print([f'osp.{fn}' for fn in dir(osp) if 'rel' in fn])

['osp.relpath']


In [15]:

# Find all the notebook files with the word "color" in their name
black_list = ['.ipynb_checkpoints', '$Recycle.Bin']
folder_path = '../../color/ipynb'
ascii_regex = re.compile(r'[^A-Za-z0-9]+')
file_ending = '.ipynb'
file_paths_list = []
for root_dir in ['C:\\', 'D:\\']:
    for parent_directory, child_directories, files_list in os.walk(root_dir):
        if all(map(lambda x: x not in parent_directory, black_list)):
            for old_file_name in files_list:
                if old_file_name.endswith(file_ending) and ('color' in old_file_name.lower()):
                    old_file_path = os.path.join(parent_directory, old_file_name)
                    file_paths_list.append(old_file_path)

In [24]:

# Copy all the notebook files with the word "color" in their name to the color repo
import shutil

for old_file_path in file_paths_list:
    new_file_name = ascii_regex.sub('_', osp.relpath(old_file_path, osp.abspath(folder_path)).replace(file_ending, '').lstrip('.\\')) + file_ending
    new_file_path = osp.abspath(osp.join(folder_path, new_file_name))
    shutil.copy(old_file_path, new_file_path)

In [9]:

print_strs_list = ['']
print_strs_list.append('# Exclude everything except text-based OC or personal directories')
print_strs_list.append('/*')
github_folder = osp.dirname(osp.abspath(osp.curdir))
# print(github_folder)

# Get the list of all items in the parent folder and filter in only the directories
black_list = ['.ipynb_checkpoints', '$Recycle.Bin', '.git', 'data', 'saves', '__pycache__', 'LEET-CODE']
for parent_directory in os.listdir(github_folder):
    if all(map(lambda x: x not in parent_directory, black_list)) and osp.isdir(osp.join(github_folder, parent_directory)):
        print_strs_list.append(f'!/{parent_directory}')
print('\n'.join(print_strs_list))


# Exclude everything except text-based OC or personal directories
/*
!/algorithms
!/builders
!/cell_techniques
!/coin-analytics
!/curve_fitting
!/ipynb
!/json
!/library_exploration
!/load_magic
!/log
!/maintenance
!/mathematics
!/programming_languages
!/ps1
!/py
!/r_programming_language
!/statistics
!/tests
!/text_manipulation
!/visualizations
!/wheels


In [ ]:

file_path = '../data/txt/resume.txt'
nu.open_path_in_notepad(file_path)

In [ ]:

black_list = ['.ipynb_checkpoints', '$Recycle.Bin']
# walk_folder = osp.dirname(nu.github_folder)
for parent_directory, child_directories, files_list in os.walk('C:\\'):
    if all(map(lambda x: x not in parent_directory, black_list)):
        for file_name in files_list:
            if ('hair' in file_name):
                file_path = osp.join(parent_directory, file_name)
                print(file_path)

In [4]:

for parent_directory, child_directories, files_list in os.walk('C:\\'):
    for file_name in files_list:
        if file_name.endswith('.docx') and ('resume' in file_name.lower()):
            file_path = osp.join(parent_directory, file_name)
            if osp.isfile(file_path):
                print(f'{file_path}')

C:\Users\daveb\OneDrive\Documents\Babbitt Stuff\Jay_s Junk\Jay-Babbitt_Resume.docx
C:\Users\daveb\OneDrive\Documents\Google Drive\Other\Job Hunting\Dave_Babbitt_Resume (1).docx
C:\Users\daveb\OneDrive\Documents\Google Drive\Other\Job Hunting\Dave_Babbitt_Resume.docx
C:\Users\daveb\OneDrive\Documents\Google Drive\Other\Job Hunting\Dave_Babbitt_Resume_Cold_Fusion_Developer.docx
C:\Users\daveb\OneDrive\Documents\Google Drive\Other\Job Hunting\Dave_Babbitt_Resume_Delivery_Driver.docx
C:\Users\daveb\OneDrive\Documents\Google Drive\Other\Job Hunting\Dave_Babbitt_Resume_Jr._Java_Developer.docx
C:\Users\daveb\OneDrive\Documents\Google Drive\Other\Job Hunting\Dave_Babbitt_Resume_Predictive_Analyst.docx
C:\Users\daveb\OneDrive\Documents\Google Drive\Other\Job Hunting\Dave_Babbitt_Resume_QA_Tester.docx
C:\Users\daveb\OneDrive\Documents\Google Drive\Other\Job Hunting\Dave_Babbitt_Resume_Reactor_Operator.docx
C:\Users\daveb\OneDrive\Documents\Google Drive\Other\Job Hunting\Dave_Babbitt_Resume_Resea

In [13]:

# Get a list of rogue functions already in scraping utilities
scraping_utils_set = nu.get_utility_file_functions(util_path='../py/scraping_utils.py')

# Make a set of rogue su functions
fn_regex = re.compile(r'\s+"def ([a-z0-9_]+)\(')
black_list = ['.ipynb_checkpoints', '$Recycle.Bin']
rows_list = []
walk_folder = osp.dirname(osp.abspath(osp.curdir))
rogue_su_fns_set = set()
for parent_directory, child_directories, files_list in os.walk(walk_folder):
    if all(map(lambda x: x not in parent_directory, black_list)):
        for file_name in files_list:
            if file_name.endswith('.ipynb'):
                file_path = osp.join(parent_directory, file_name)
                with open(file_path, 'r', encoding='utf-8') as f:
                    lines_list = f.readlines()
                    for line in lines_list:
                        match_obj = fn_regex.search(line)
                        if match_obj:
                            fn = match_obj.group(1)
                            if fn in scraping_utils_set: rogue_su_fns_set.add(fn)
if rogue_su_fns_set:
    print(f'Search for *.ipynb; file masks in the {walk_folder} folder for this pattern:')
    print('\\s+"def (' + '|'.join(rogue_su_fns_set) + ')\(')
    print('Replace each of the calls to these definitions with calls the the su. equivalent (and delete the definitions).')

In [12]:

# Get a list of multi-notebook functions as notebook utilities candidates
fn_regex = re.compile(r'\s+"def ([a-z0-9_]+)\(')
black_list = ['.ipynb_checkpoints', '$Recycle.Bin']
rows_list = []
walk_folder = osp.dirname(osp.abspath(osp.curdir))
for parent_directory, child_directories, files_list in os.walk(walk_folder):
    if all(map(lambda x: x not in parent_directory, black_list)):
        for file_name in files_list:
            if file_name.endswith('.ipynb'):
                file_path = osp.join(parent_directory, file_name)
                with open(file_path, 'r', encoding='utf-8') as f:
                    lines_list = f.readlines()
                    fns_set = set()
                    for line in lines_list:
                        match_obj = fn_regex.search(line)
                        if match_obj:
                            fn = match_obj.group(1)
                            fns_set.add(fn)
                    for fn in fns_set:
                        row_dict = {}
                        row_dict['file_name'] = file_name
                        row_dict['function_name'] = fn
                        rows_list.append(row_dict)
function_names_df = pd.DataFrame(rows_list)
df = function_names_df.groupby('function_name').count().sort_values('file_name', ascending=False)
mask_series = (df.file_name > 1) & df.index.map(lambda x: len(x) > 4)
fns_list = df[mask_series].index.tolist()
if fns_list:
    print(f'Search for *.ipynb; file masks in the {walk_folder} folder for this pattern:')
    print('\\s+"def (' + '|'.join(fns_list) + ')\(')
    print('Find a way to get rid of these definition duplicates.')

In [7]:

import os

for parent_directory, child_directories, files_list in os.walk(r'D:\\'):
    for file_name in files_list:
        if file_name.endswith('.svg'):
            file_path = osp.join(parent_directory, file_name)
            if osp.isfile(file_path):
                with open(file_path, 'r', errors='ignore') as file:
                    data = file.read()
                    if 'Retarded' in data:
                        print(f'{file_path}')

D:\\Geek Squad Data Backup\HDD\Documents\GitHub\notebooks_old\political_triangle\svg\nick_land_comments.svg
D:\\Geek Squad Data Backup\HDD\Documents\GitHub\notebooks_old\political_triangle\svg\nick_land_comments_copy.svg
D:\\Geek Squad Data Backup\HDD\Documents\GitHub\notebooks_old\political_triangle\svg\nick_land_comments_optimized.svg
D:\\Geek Squad Data Backup\HDD\Documents\GitHub\notebooks_old\political_triangle\svg\nick_land_comments_plain.svg
D:\\Geek Squad Data Backup\HDD\Documents\GitHub\notebooks_old\political_triangle\svg\.ipynb_checkpoints\nick_land_comments_optimized-checkpoint.svg
